In [8]:
import datetime

import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dropout, Flatten, Dense

# Imports for the HParams plugin
from tensorboard.plugins.hparams import api_pb2
from tensorboard.plugins.hparams import summary as hparams_summary
from google.protobuf import struct_pb2

In [9]:
# Get MNIST dataset
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
test_images = test_images.reshape((10000, 28, 28, 1))

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

# Convolutional Neural Network

Network architect (from input to output):
* Conv2D
    * Number of kernels: 32
    * Kernel size: hyperparamters
* Conv2D
    * Number of kernels: 64
    * Kernel size: hyperparamters
* Maxpooling
    * Kernel size: (2, 2)
* Dropout
* Dense
    * Number of units: hyperparameters
* Dropout
* Dense
    * Number of units: 10

With the overal architect being kept constant, the following hyperparameters are tuned:
1. Learning rate
2. Size of kernel used in Conv2D layers (2 Conv2D layers use kernels of the same size)
3. Dropout rate
4. Number of units on the first Dense layer


In [10]:
# Set up hyperparameters lists
learning_rate_list = [1e-3, 1e-2, 1e-1]
kernel_size_list = [3, 5]
dropout_rate_list = [0., 0.25, 0.5]
dense_units_list = [32, 128]

In [11]:
# Add hyperparamters to Hparam 
def create_experiment_summary(learning_rate_list, dropout_rate_list, kernel_size_list, dense_units_list):
    learning_rate_list_val = struct_pb2.ListValue()
    learning_rate_list_val.extend(learning_rate_list)
    
    dropout_rate_list_val = struct_pb2.ListValue()
    dropout_rate_list_val.extend(dropout_rate_list)
    
    kernel_size_list_val = struct_pb2.ListValue()
    kernel_size_list_val.extend(kernel_size_list)
    
    dense_units_list_val = struct_pb2.ListValue()
    dense_units_list_val.extend(dense_units_list)
    
    return hparams_summary.experiment_pb(
      # The hyperparameters being changed
      hparam_infos=[
          api_pb2.HParamInfo(name='learning_rate',
                             display_name='Learning Rate',
                             type=api_pb2.DATA_TYPE_FLOAT64,
                             domain_discrete=learning_rate_list_val),
          api_pb2.HParamInfo(name='kernel_size',
                             display_name='Kernel Size',
                             type=api_pb2.DATA_TYPE_FLOAT64,
                             domain_discrete=kernel_size_list_val),
          api_pb2.HParamInfo(name='dropout_rate',
                             display_name='Dropout rate',
                             type=api_pb2.DATA_TYPE_FLOAT64,
                             domain_discrete=dropout_rate_list_val),
          api_pb2.HParamInfo(name='dense_units',
                             display_name='Dense Units',
                             type=api_pb2.DATA_TYPE_FLOAT64,
                             domain_discrete=dense_units_list_val)
      ],
      # The metrics being tracked
      metric_infos=[
          api_pb2.MetricInfo(
              name=api_pb2.MetricName(
                  tag='accuracy'),
              display_name='Accuracy'),
      ]
    )

exp_summary = create_experiment_summary(learning_rate_list, dropout_rate_list, kernel_size_list, dense_units_list)
root_logdir_writer = tf.summary.create_file_writer("logs_cnn/hparam_tuning")
with root_logdir_writer.as_default():
    tf.summary.import_event(tf.compat.v1.Event(summary=exp_summary).SerializeToString())

Define model and adapt it to be trained with hyperparameters fed in form of dictionary

In [12]:
def train_cnn(hparams):
    model = tf.keras.models.Sequential()
    model.add(Conv2D(32, kernel_size=hparams['kernel_size'],
                 activation='relu',
                 input_shape=(28, 28, 1)))
    
    model.add(Conv2D(64, 
                     kernel_size=hparams['kernel_size'], 
                     activation='relu'))
    
    model.add(MaxPool2D(pool_size=(2, 2)))
    
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    
    model.add(Dense(hparams['dense_units'], activation='relu'))
    
    model.add(Dropout(hparams['dropout_rate']))
    
    model.add(Dense(10, activation='softmax'))
    
    optim = tf.keras.optimizers.Adam(lr=hparams['learning_rate'])
    
    model.compile(optimizer=optim,
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

    model.fit(train_images, 
              train_labels,
              batch_size=64,
              epochs=1) # Run with 1 epoch to speed things up for demo purposes
    
    _, accuracy = model.evaluate(test_images, test_labels)
    return accuracy
    

For each run, log an hparams summary with the hyperparameters and final accuracy

In [13]:
def run(run_dir, hparams):
    writer = tf.summary.create_file_writer(run_dir)
    summary_start = hparams_summary.session_start_pb(hparams=hparams)

    with writer.as_default():
        accuracy = train_cnn(hparams)
        summary_end = hparams_summary.session_end_pb(api_pb2.STATUS_SUCCESS)

        tf.summary.scalar('accuracy', accuracy, step=1, description="The accuracy")
        tf.summary.import_event(tf.compat.v1.Event(summary=summary_start).SerializeToString())
        tf.summary.import_event(tf.compat.v1.Event(summary=summary_end).SerializeToString())

# Start trainning and log the accuracy

In [14]:
session_num = 0

for learning_rate in learning_rate_list:
    for kernel_size in kernel_size_list:
        for dense_units in dense_units_list:
            for dropout_rate in dropout_rate_list:
                hparams = {'learning_rate':learning_rate,
                           'kernel_size': kernel_size,
                           'dropout_rate': dropout_rate,
                           'dense_units': dense_units
                          }
                print('--- Running training session %d' % (session_num + 1))
                print(hparams)
                run_name = "run-%d" % session_num
                run("logs_cnn/hparam_tuning/" + run_name, hparams)
                session_num += 1

--- Running training session 1
{'learning_rate': 0.001, 'kernel_size': 3, 'dropout_rate': 0, 'dense_units': 32}
10000/10000 [==============================] - 2s 191us/sample - loss: 0.0516 - accuracy: 0.9843
--- Running training session 2
{'learning_rate': 0.001, 'kernel_size': 3, 'dropout_rate': 0.25, 'dense_units': 32}
10000/10000 [==============================] - 2s 170us/sample - loss: 0.0633 - accuracy: 0.9808
--- Running training session 3
{'learning_rate': 0.001, 'kernel_size': 3, 'dropout_rate': 0.5, 'dense_units': 32}
10000/10000 [==============================] - 2s 172us/sample - loss: 0.0798 - accuracy: 0.9756
--- Running training session 4
{'learning_rate': 0.001, 'kernel_size': 3, 'dropout_rate': 0, 'dense_units': 128}
10000/10000 [==============================] - 2s 200us/sample - loss: 0.0403 - accuracy: 0.9867
--- Running training session 5
{'learning_rate': 0.001, 'kernel_size': 3, 'dropout_rate': 0.25, 'dense_units': 128}
10000/10000 [=============================

10000/10000 [==============================] - 2s 174us/sample - loss: 2.3047 - accuracy: 0.1135
--- Running training session 28
{'learning_rate': 0.1, 'kernel_size': 3, 'dropout_rate': 0, 'dense_units': 128}
10000/10000 [==============================] - 2s 178us/sample - loss: 2.3045 - accuracy: 0.1028
--- Running training session 29
{'learning_rate': 0.1, 'kernel_size': 3, 'dropout_rate': 0.25, 'dense_units': 128}
10000/10000 [==============================] - 2s 176us/sample - loss: 2.3128 - accuracy: 0.1028
--- Running training session 30
{'learning_rate': 0.1, 'kernel_size': 3, 'dropout_rate': 0.5, 'dense_units': 128}
10000/10000 [==============================] - 2s 177us/sample - loss: 2.3089 - accuracy: 0.0982
--- Running training session 31
{'learning_rate': 0.1, 'kernel_size': 5, 'dropout_rate': 0, 'dense_units': 32}
10000/10000 [==============================] - 2s 204us/sample - loss: 2.3110 - accuracy: 0.1135
--- Running training session 32
{'learning_rate': 0.1, 'kernel_

In [2]:
# (train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

# train_images = train_images.reshape((60000, 28, 28, 1))
# test_images = test_images.reshape((10000, 28, 28, 1))

# # Normalize pixel values to be between 0 and 1
# train_images, test_images = train_images / 255.0, test_images / 255.0

In [8]:
# model = tf.keras.models.Sequential()
# model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
# model.add(tf.keras.layers.MaxPooling2D((2, 2)))
# model.add(Conv2D(64, (3, 3), activation='relu'))
# model.add(tf.keras.layers.MaxPooling2D((2, 2)))
# model.add(Conv2D(64, (3, 3), activation='relu'))
# model.add(tf.keras.layers.Flatten())
# model.add(tf.keras.layers.Dense(64, activation='relu'))
# model.add(tf.keras.layers.Dense(10, activation='softmax'))
# model.summary()


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 576)               0         
_________________________________________________________________
dense (Dense)                (None, 64)               

In [9]:
# model.compile(optimizer='adam',
#               loss='sparse_categorical_crossentropy',
#               metrics=['accuracy'])

# model.fit(train_images, train_labels, epochs=1)

60000/60000 [==============================] - 16s 272us/sample - loss: 0.1445 - accuracy: 0.9549
